In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.datasets import OxfordIIITPet
# import matplotlib.pyplot as plt
import segmentation_models_pytorch as smp
from PIL import Image

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [2]:
class OxfordIIITPetsAugmented(OxfordIIITPet):
    """inherit from torchvision.datasets.OxfordIIITPet"""
    def __init__(self,
                 root: str,
                 split: str,
                 target_types="segmentation",
                 download=True,
                 pre_transform=None,
                 post_transform=None,
                 pre_target_transform=None,
                 post_target_transform=None, 
                 common_transform=None):
        
        super().__init__(
            root=root,
            split=split,
            target_types=target_types,
            download=download,
            transform=pre_transform,
            target_transform=pre_target_transform
        )
        
        self.post_transform = post_transform
        self.post_target_transform = post_target_transform
        self.common_transform = common_transform

    def __getitem__(self, idx):
        input, target = super().__getitem__(idx)
        
        if self.common_transform is not None:
            both = torch.cat([input, target], dim=0)
            both = self.common_transform(both)
            input, target = torch.split(both, 3, dim=0)
            
        if self.post_transform is not None:
            input = self.post_transform(input)
            
        if self.post_target_transform is not None:
            target = self.post_target_transform(target)
            
        return input, target

In [3]:
def tensor_trimap(t):
    x = t * 255
    x = x.to(torch.long)
    x = x - 1
    return x

def get_oxford_dataloaders(root='./OxfordPetsData', train_size=64, test_size=32):
    transform_dict = {
        'pre_transform': transforms.ToTensor(),
        'pre_target_transform': transforms.ToTensor(),
        'common_transform': transforms.Compose([
            transforms.Resize((224, 224), interpolation=transforms.InterpolationMode.NEAREST),
            transforms.RandomHorizontalFlip(p=0.5),
        ]),
        'post_transform': transforms.Compose([
            transforms.ColorJitter(contrast=0.3),
        ]),
        'post_target_transform': transforms.Compose([
            transforms.Lambda(tensor_trimap),
        ])
    }


    train_dataset = OxfordIIITPetsAugmented(
        root=root,
        split="trainval",
        target_types="segmentation",
        download=True,  
        **transform_dict
    )

   
    test_dataset = OxfordIIITPetsAugmented(
        root=root,  
        split="test",
        target_types="segmentation",
        download=True,
        **transform_dict
    )

    train_loader = DataLoader(
        train_dataset,
        batch_size=train_size,
        shuffle=True,
    )

    test_loader = DataLoader(
        test_dataset,
        batch_size=test_size,
        shuffle=False,
    )

    return train_dataset, test_dataset, train_loader, test_loader

In [4]:
train_ds, test_ds, train_loader, test_loader = get_oxford_dataloaders(train_size=32, test_size=16)

In [5]:
for images, masks in train_loader:
    print(f"Image batch shape: {images.shape}")  # [batch_size, 3, 224, 224]
    print(f"Mask batch shape: {masks.shape}")  # [batch_size, 1, 224, 224]
    break  

Image batch shape: torch.Size([32, 3, 224, 224])
Mask batch shape: torch.Size([32, 1, 224, 224])


In [5]:
def UNet():
    model = smp.Unet(
        encoder_name="resnet34",
        encoder_weights="imagenet",
        in_channels=3,
        classes=3,
    )
    return model

def dice_coefficient(y_true, y_pred):
    """
    y_true: 真实标签 (B, H, W)
    y_pred: 预测标签 (B, H, W)
    """
    
    num_classes = 3  
    dice_scores = []
    smooth = 1e-7  

    for class_id in range(num_classes):
        true_flat = (y_true == class_id).float()
        pred_flat = (y_pred == class_id).float()
        
        intersection = torch.sum(true_flat * pred_flat, dim=[1,2])
        union = torch.sum(true_flat, dim=[1,2]) + torch.sum(pred_flat, dim=[1,2])
        
        dice = (2.0 * intersection + smooth) / (union + smooth)
        dice_scores.append(dice.mean())
    
    return torch.mean(torch.stack(dice_scores))

In [7]:
from focalloss import focal_loss

In [6]:
from torchvision.utils import make_grid
from PIL import Image, ImageDraw, ImageFont
import os

def visualize_predictions(model, val_loader, epoch, model_name):
    device = next(model.parameters()).device
    model.eval()
    # model_name = str(model)
    
    with torch.no_grad():
        samples = []
        for i, (images, masks) in enumerate(val_loader):
            if i >= 3:
                break
            images = images.to(device)
            masks = masks.squeeze(1).to(device)
            outputs = model(images)
            predictions = torch.argmax(outputs, dim=1)
            
            samples.append((
                images[0:1],  # [1, C, H, W]
                masks[0:1].unsqueeze(1).repeat(1,3,1,1),  # [1, 3, H, W]
                predictions[0:1].unsqueeze(1).repeat(1,3,1,1)  # [1, 3, H, W]
            ))
            
        img_row = torch.cat([s[0] for s in samples], dim=0)  # [3, C, H, W]
        mask_row = torch.cat([s[1] for s in samples], dim=0)  # [3, C, H, W] 
        pred_row = torch.cat([s[2] for s in samples], dim=0)  # [3, C, H, W]
        
        grid = make_grid(
            torch.cat([img_row, mask_row, pred_row], dim=0),
            nrow=3,
            normalize=True,
            pad_value=1,
            padding=2
        )
        
        #grid_np = grid.permute(1, 2, 0).cpu().numpy()
        # plt.imsave(f'./images/epoch_{epoch}.jpg', grid_np)

        grid_np = (grid.permute(1, 2, 0).cpu().numpy() * 255).astype('uint8')
        img = Image.fromarray(grid_np)
        # Image.fromarray(grid_np).resize((224, 224), Image.Resampling.LANCZOS).save(f'./images/epoch_{epoch}.jpg', quality=90)
        # save_dir = os.path.join("./images/", model_name, "/epoch_{epoch}.jpg")
        # Image.fromarray(grid_np).resize((224, 224), Image.Resampling.LANCZOS).save(save_dir, quality=90)

        title_height = 50
        new_img = Image.new('RGB', (img.width, img.height + title_height), 'white')
        draw = ImageDraw.Draw(new_img)
        
        try:
            font = ImageFont.truetype("arial.ttf", 24)
        except:
            font = ImageFont.load_default()
            
        titles = ['Input', 'Ground Truth', 'Prediction']
        width_per_img = img.width // 3
        for i, title in enumerate(titles):
            w = draw.textlength(title, font=font)
            x = i * width_per_img + (width_per_img - w) // 2
            draw.text((x, 10), title, font=font, fill='black')

        save_dir = os.path.join("./images/", model_name)
        os.makedirs(save_dir, exist_ok=True)
        save_path = os.path.join(save_dir, f"epoch_{epoch}.jpg")
            
        new_img.paste(img, (0, title_height))
        new_img.resize((512, 512), Image.Resampling.LANCZOS).save(save_path, quality=90)

        # save_dir = os.path.join("./images/", model_name)
        # os.makedirs(save_dir, exist_ok=True)
        # save_path = os.path.join(save_dir, f"epoch_{epoch}.jpg")
        # Image.fromarray(grid_np).resize((512, 512), Image.Resampling.LANCZOS).save(save_path, quality=90)

In [7]:
from tqdm import tqdm
from torchmetrics import JaccardIndex

jaccard = JaccardIndex(task="multiclass", num_classes=3).to(device)

def train_model(model, train_loader, val_loader, epochs=20, learning_rate=5e-4, model_name = "model"):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=0.0001)
    scheduler = torch.optim.lr_scheduler.LinearLR(
        optimizer,
        start_factor=1.0,
        end_factor=0.01,
        total_iters=epochs
        )
    
    os.makedirs('./checkpoints', exist_ok=True)
    best_val_loss = float('inf')
    
    for epoch in range(epochs):
        # try:
        #     visualize_predictions(model, val_loader, epoch)
        # except Exception as e:
        #     print(f"Visualization failed: {str(e)}")
        
        # Training phase
        if epoch % 2 == 0:
            visualize_predictions(model, val_loader, epoch=epoch, model_name=model_name)

        model.train()
        train_loss = 0
        train_dice = 0
        train_acc = 0
        train_miou = 0
        n_batches = 0
        
        pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs} [Train]')
        for images, masks in pbar:
            images = images.to(device)
            masks = masks.squeeze(1).to(device)
            
            outputs = model(images)
            loss = criterion(outputs, masks)
            
            preds = torch.argmax(outputs, dim=1)
            dice = dice_coefficient(masks, preds)
            accuracy = (preds == masks).float().mean()
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            train_dice += dice.item()
            train_acc += accuracy.item()
            train_miou += jaccard(preds, masks).item()
            n_batches += 1
            
            avg_loss = train_loss / n_batches
            avg_dice = train_dice / n_batches
            avg_acc = train_acc / n_batches
            avg_miou = train_miou / n_batches
            
            pbar.set_postfix({
                'loss': f"{avg_loss:.4f}",
                'miou': f"{avg_miou:.4f}",
                'dice': f"{avg_dice:.4f}",
                'pixel acc': f"{avg_acc:.4f}"
            })

        scheduler.step()
        
        # Validation phase
        model.eval()
        val_loss = 0
        val_dice = 0
        val_acc = 0
        val_miou = 0
        n_batches = 0
        
        # pbar = tqdm(val_loader, desc=f'Epoch {epoch+1}/{epochs} [Valid]')
        with torch.no_grad():
            for images, masks in val_loader:
                images = images.to(device)
                masks = masks.squeeze(1).to(device)
                
                outputs = model(images)
                loss = criterion(outputs, masks)
                
                preds = torch.argmax(outputs, dim=1)
                dice = dice_coefficient(masks, preds)
                accuracy = (preds == masks).float().mean()
                
                val_loss += loss.item()
                val_dice += dice.item()
                val_acc += accuracy.item()
                val_miou += jaccard(preds, masks).item()
                n_batches += 1
                
        avg_val_loss = val_loss / n_batches
        avg_val_dice = val_dice / n_batches
        avg_val_acc = val_acc / n_batches
        avg_val_miou = val_miou / n_batches
                
                # pbar.set_postfix({
                #     'val_loss': f"{avg_val_loss:.4f}",
                #     'val_dice': f"{avg_val_dice:.4f}",
                #     'val_acc': f"{avg_val_acc:.4f}"
                # })
        # pbar.set_description(f"Val Loss: {avg_val_loss:.4f}, Val dice: {avg_val_dice:.4f}, Val acc: {avg_val_acc:.4f}")
        print(f"[Valid] Loss: {avg_val_loss:.4f}, Dice: {avg_val_dice:.4f}, Pixel acc: {avg_val_acc:.4f}, mIoU: {avg_val_miou:.4f}, LR: {scheduler.get_last_lr()[0]:.6f}")

        # Save best model
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_loss': avg_val_loss,
                'val_dice': avg_val_dice,
                'val_acc': avg_val_acc,
            }, './checkpoints/best_unet.pth')
    
    # return model

In [42]:
unet = UNet()
train_model(unet, train_loader, test_loader)

Epoch 1/20 [Train]: 100%|██████████| 115/115 [00:48<00:00,  2.36it/s, loss=0.3547, miou=0.6794, dice=0.7740, acc=0.8698]


[Valid] Loss: 0.3002, Dice: 0.8010, Acc: 0.8910, mIoU: 0.7083


Epoch 2/20 [Train]: 100%|██████████| 115/115 [00:54<00:00,  2.10it/s, loss=0.2355, miou=0.7488, dice=0.8337, acc=0.9104]


[Valid] Loss: 0.2315, Dice: 0.8422, Acc: 0.9136, mIoU: 0.7582


Epoch 3/20 [Train]: 100%|██████████| 115/115 [00:53<00:00,  2.13it/s, loss=0.2144, miou=0.7635, dice=0.8448, acc=0.9174]


...

[Valid] Loss: 0.2684, Dice: 0.8377, Acc: 0.9103, mIoU: 0.7547


Epoch 19/20 [Train]: 100%|██████████| 115/115 [00:54<00:00,  2.12it/s, loss=0.1663, miou=0.8006, dice=0.8713, acc=0.9338]


[Valid] Loss: 0.2362, Dice: 0.8523, Acc: 0.9158, mIoU: 0.7712


Epoch 20/20 [Train]: 100%|██████████| 115/115 [00:54<00:00,  2.12it/s, loss=0.1382, miou=0.8215, dice=0.8859, acc=0.9432]


[Valid] Loss: 0.2225, Dice: 0.8639, Acc: 0.9230, mIoU: 0.7865


In [33]:
checkpoint = torch.load('./checkpoints/best_unet.pth')
unet = UNet()
unet.load_state_dict(checkpoint['model_state_dict'])

visualize_predictions(unet, test_loader, epoch=0)

In [7]:
def focal_loss(alpha=0.25, gamma=1.2):
    def loss(y_true, y_pred):
        epsilon = 1e-7
        y_pred = tf.clip_by_value(y_pred, epsilon, 1. - epsilon)
        ce = -y_true * tf.math.log(y_pred)
        weights = tf.pow(1 - y_pred, gamma) * y_true
        weights = weights * alpha
        fl = weights * ce
        return tf.reduce_mean(tf.reduce_sum(fl, axis=-1))  
    return loss

class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name="f1_score", **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.precision = tf.keras.metrics.Precision()
        self.recall = tf.keras.metrics.Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.argmax(y_true, axis=-1)
        y_pred = tf.argmax(y_pred, axis=-1)
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        precision = self.precision.result()
        recall = self.recall.result()
        return 2 * (precision * recall) / (precision + recall + 1e-7)  

    def reset_states(self):
        self.precision.reset_states()
        self.recall.reset_states()

In [75]:
# Transformer Encoder UNet
# def TEUNet():
#     model = smp.UnetPlusPlus(  
#         encoder_name="mit_b2",
#         encoder_weights="imagenet",
#         in_channels=3,
#         classes=3
#     )
#     return model

def UNet50():
    model = smp.Unet(
        encoder_name="resnet50",
        encoder_weights="imagenet",
        in_channels=3,
        classes=3,
    )
    return model

def DenseUNet():
   model = smp.UnetPlusPlus(
       encoder_name="densenet121",  #densenet169, densenet201, densenet161
       encoder_weights="imagenet",
       in_channels=3,
       classes=3
   )
   return model

def FPN():
    model = smp.FPN(
        encoder_name="resnet34",
        encoder_weights="imagenet",
        in_channels=3,
        classes=3
    )
    return model

def DeepLabV3():
    model = smp.DeepLabV3(
        encoder_name="resnet34", 
        encoder_weights="imagenet",
        in_channels=3,
        classes=3
    )
    return model

In [74]:
fpn = FPN()
train_model(fpn, train_loader, test_loader, epochs=20, learning_rate=1e-4, model_name="fpn")

Epoch 1/20 [Train]: 100%|██████████| 115/115 [00:40<00:00,  2.87it/s, loss=0.5033, miou=0.5982, dice=0.6739, acc=0.8318]


[Valid] Loss: 0.2572, Dice: 0.8185, Acc: 0.9028, mIoU: 0.7330


Epoch 2/20 [Train]: 100%|██████████| 115/115 [00:46<00:00,  2.47it/s, loss=0.2609, miou=0.7179, dice=0.7983, acc=0.8992]


[Valid] Loss: 0.2308, Dice: 0.8356, Acc: 0.9152, mIoU: 0.7546


Epoch 3/20 [Train]: 100%|██████████| 115/115 [00:44<00:00,  2.56it/s, loss=0.2164, miou=0.7540, dice=0.8316, acc=0.9155]


...

[Valid] Loss: 0.2233, Dice: 0.8604, Acc: 0.9249, mIoU: 0.7843


Epoch 19/20 [Train]: 100%|██████████| 115/115 [00:47<00:00,  2.43it/s, loss=0.1201, miou=0.8388, dice=0.8968, acc=0.9500]


[Valid] Loss: 0.2146, Dice: 0.8708, Acc: 0.9304, mIoU: 0.7978


Epoch 20/20 [Train]: 100%|██████████| 115/115 [00:47<00:00,  2.44it/s, loss=0.1110, miou=0.8480, dice=0.9039, acc=0.9535]


[Valid] Loss: 0.2212, Dice: 0.8678, Acc: 0.9300, mIoU: 0.7941


In [80]:
unet50 = UNet50()
train_model(unet50, train_loader, test_loader, epochs=20, learning_rate=1e-4, model_name="unet50")

Epoch 1/20 [Train]: 100%|██████████| 115/115 [01:11<00:00,  1.62it/s, loss=0.6610, miou=0.6247, dice=0.7241, pixel acc=0.8077]


[Valid] Loss: 0.4102, Dice: 0.8459, Pixel acc: 0.9115, mIoU: 0.7627, LR: 0.000095


Epoch 2/20 [Train]: 100%|██████████| 115/115 [01:18<00:00,  1.47it/s, loss=0.3392, miou=0.7699, dice=0.8485, pixel acc=0.9184]


[Valid] Loss: 0.2783, Dice: 0.8601, Pixel acc: 0.9245, mIoU: 0.7825, LR: 0.000090


Epoch 3/20 [Train]: 100%|██████████| 115/115 [01:17<00:00,  1.48it/s, loss=0.2478, miou=0.7926, dice=0.8657, pixel acc=0.9297]


...

[Valid] Loss: 0.2022, Dice: 0.8771, Pixel acc: 0.9338, mIoU: 0.8057, LR: 0.000011


Epoch 19/20 [Train]: 100%|██████████| 115/115 [01:16<00:00,  1.49it/s, loss=0.0861, miou=0.8853, dice=0.9294, pixel acc=0.9661]


[Valid] Loss: 0.2049, Dice: 0.8758, Pixel acc: 0.9334, mIoU: 0.8038, LR: 0.000006


Epoch 20/20 [Train]: 100%|██████████| 115/115 [01:14<00:00,  1.54it/s, loss=0.0843, miou=0.8873, dice=0.9311, pixel acc=0.9668]


[Valid] Loss: 0.2038, Dice: 0.8763, Pixel acc: 0.9338, mIoU: 0.8045, LR: 0.000001


In [81]:
deeplab = DeepLabV3()
train_model(deeplab, train_loader, test_loader, epochs=20, learning_rate=5e-4, model_name="deeplab")

Epoch 1/20 [Train]: 100%|██████████| 115/115 [01:13<00:00,  1.56it/s, loss=0.3381, miou=0.6539, dice=0.7398, pixel acc=0.8675]


[Valid] Loss: 0.3385, Dice: 0.7720, Pixel acc: 0.8723, mIoU: 0.6779, LR: 0.000475


Epoch 2/20 [Train]: 100%|██████████| 115/115 [01:17<00:00,  1.48it/s, loss=0.2285, miou=0.7479, dice=0.8298, pixel acc=0.9113]


[Valid] Loss: 0.2307, Dice: 0.8355, Pixel acc: 0.9112, mIoU: 0.7519, LR: 0.000450


Epoch 3/20 [Train]: 100%|██████████| 115/115 [01:18<00:00,  1.47it/s, loss=0.2070, miou=0.7632, dice=0.8424, pixel acc=0.9182]


...

[Valid] Loss: 0.2122, Dice: 0.8717, Pixel acc: 0.9315, mIoU: 0.8000, LR: 0.000055


Epoch 19/20 [Train]: 100%|██████████| 115/115 [01:18<00:00,  1.46it/s, loss=0.0882, miou=0.8744, dice=0.9220, pixel acc=0.9626]


[Valid] Loss: 0.2146, Dice: 0.8714, Pixel acc: 0.9316, mIoU: 0.7995, LR: 0.000030


Epoch 20/20 [Train]: 100%|██████████| 115/115 [01:18<00:00,  1.46it/s, loss=0.0860, miou=0.8771, dice=0.9237, pixel acc=0.9635]


[Valid] Loss: 0.2181, Dice: 0.8716, Pixel acc: 0.9315, mIoU: 0.7998, LR: 0.000005


In [8]:
def DeepLabV3_50():
    model = smp.DeepLabV3(
        encoder_name="resnet50", 
        encoder_weights="imagenet",
        in_channels=3,
        classes=3
    )
    return model

deeplab50 = DeepLabV3_50()
train_model(deeplab50, train_loader, test_loader, epochs=15, learning_rate=1e-4, model_name="deeplab50")

Epoch 1/15 [Train]: 100%|██████████| 115/115 [18:25<00:00,  9.61s/it, loss=0.3270, miou=0.6624, dice=0.7447, pixel acc=0.8739]


[Valid] Loss: 0.2307, Dice: 0.8308, Pixel acc: 0.9123, mIoU: 0.7497, LR: 0.000093


Epoch 2/15 [Train]: 100%|██████████| 115/115 [18:11<00:00,  9.49s/it, loss=0.2039, miou=0.7676, dice=0.8441, pixel acc=0.9206]


[Valid] Loss: 0.2033, Dice: 0.8536, Pixel acc: 0.9226, mIoU: 0.7765, LR: 0.000087


Epoch 3/15 [Train]: 100%|██████████| 115/115 [17:51<00:00,  9.32s/it, loss=0.1736, miou=0.7912, dice=0.8624, pixel acc=0.9310]


...

[Valid] Loss: 0.1920, Dice: 0.8745, Pixel acc: 0.9330, mIoU: 0.8035, LR: 0.000014


Epoch 14/15 [Train]: 100%|██████████| 115/115 [18:06<00:00,  9.45s/it, loss=0.0925, miou=0.8715, dice=0.9193, pixel acc=0.9615]


[Valid] Loss: 0.1950, Dice: 0.8733, Pixel acc: 0.9327, mIoU: 0.8018, LR: 0.000008


Epoch 15/15 [Train]: 100%|██████████| 115/115 [17:54<00:00,  9.34s/it, loss=0.0906, miou=0.8739, dice=0.9211, pixel acc=0.9623]


[Valid] Loss: 0.1939, Dice: 0.8747, Pixel acc: 0.9333, mIoU: 0.8038, LR: 0.000001
